# "Smooth" statistics & smooth line plot

Statistics "smooth" are calculated on the sample of two continuous variables (i.e., sample of points or lines).
It interpolates data points to create a smoother curve.

This notebook uses definitions from [DataFrame](https://kotlin.github.io/dataframe/overview.html).

## Usage

The "Smooth" statistic proves beneficial in scenarios with over-plotting or noise,
simplifying the process of identifying inherent trends and patterns.
It can also be used to make a more pretty line with a small number of points.

## Arguments

* Input (mandatory):
    - `x` — numeric sample of input points `x` coordinates
    - `y` — numeric sample of input points `y` coordinates
* Parameters (optional):
    - `method: SmoothMethod` — smoothing model:
        - `SmoothMethod.Linear(confidenceLevel: Double)` — linear model
        - `SmoothMethod.Polynomial(degree: Int, confidenceLevel: Double)` — polynomial model
        - `SmoothMethod.LOESS(span: Double, loessCriticalSize: Int, samplingSeed: Long, confidenceLevel: Double)` —
          Local Polynomial Regression model
    - `smootherPointCount: Int` — number of sampled points

### Generalized signature

The specific signature depends on the function,
but all functions related to "smooth" statistic (which will be discussed further below —
different variations of `statSmooth()`, `smoothLine()`) have approximately the same signature with the arguments above:

```
statSmoothArgs := 
   x, 
   y,
   method: SmoothMethod = SmoothMethod.LOESS(),
   smootherPointCount: Int = 100
```

The possible types of `x` and `y` depend on where a certain function is used.
They can be simply `Iterable` (`List`, `Set`, etc.) or a reference to a column in a `DataFrame`
(`String`, `ColumnAccessor`) or the `DataColumn` itself.

## Output statistics

| name      | type   | description                                          |
|-----------|--------|------------------------------------------------------|
| Stat.x    | Double | `x` coordinate                                       |
| Stat.y    | Double | `y` coordinate                                       |
| Stat.yMin | Double | Lower point-wise confidence interval around the mean |
| Stat.yMax | Double | Upper point-wise confidence interval around the mean |
| Stat.se   | Double | Standard error                                       |

## StatSmooth plots

In [1]:
%use dataframe
%use kandy

import org.apache.commons.math3.distribution.NormalDistribution
import org.apache.commons.math3.distribution.UniformRealDistribution
import kotlin.random.Random

In [2]:
// To generate the data, we use a standard java math library
// https://commons.apache.org/proper/commons-math/

// Generate line with formula
val xs = (-100..100).map { it / 50.0 }
val lineFormula = { x: Double -> 2.0 / (x * x + 0.5) }
// Generate noises from normal distribution
val noises = NormalDistribution(0.0, 0.1).sample(xs.size).toList()
val ys = xs.zip(noises).map { lineFormula(it.first) + it.second }
// And drop 2/3 points
val random = Random(42)
val (newXs, newYs) = xs.zip(ys).shuffled(random).take(xs.size * 1 / 3).sortedBy { it.first }.unzip()

In [3]:
// Gather them into the DataFrame
val df = dataFrameOf(
    "speed" to newXs,
    "efficiency" to newYs
)
df.head(5)

<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1073741824"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

`df` has a signature

| speed | efficiency |
|-------|------------|

Let's take a look at `StatSmooth` output DataFrame:

In [4]:
df.statSmooth("speed", "efficiency").head(5)

<iframe onload="o_resize_iframe_out_2()" style="width:100%;" class="result_container" id="iframe_out_2" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1073741825"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 1</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

It has the following signature:

<table>
  <thead>
    <tr>
      <th alignt="left" colspan="5">Stat</th>
    </tr>
  </thead>
  <thead>
    <tr>
      <th>x</th>
      <th>y</th>
      <th>yMin</th>
      <th>yMax</th>
      <th>se</th>
    </tr>
  </thead>
</table>

As you can see, we got a `DataFrame` with one `ColumnGroup` called `Stat` which contains several columns with statics.
For `statSmooth`, each row corresponds to one of the line points.
`Stat.x` is the column with this point `x` coordinate.
`Stat.y` is points `y` coordinate; `Stat.yMin` — lower point of confidence level.
`Stat.yMax` — upper point of confidence level.
`Stat.se` — standard error.

`DataFrame` with "smooth" statistics is called `StatSmoothFrame`

### `statSmooth` transform

`statSmooth(statSmoothArgs) { /*new plotting context*/ }` modifies a plotting context —
instead of original data (no matter was it empty or not) new `StatSmooth` dataset
(calculated on given arguments. Inputs can be provided as `Iterable` or as dataset column reference —
by name as a `String`, as a `ColumnReference` or as a `DataColumn`) is used inside a new context
(original dataset and primary context are not affected —
you can add layers using initial dataset outside the `statSmooth` context).
Since the old dataset is irrelevant, we cannot use references for its columns.
But we can refer to the new ones.
They are all contained in the `Stat` group and can be called inside the new context:

In [5]:
plot {
    statSmooth(newXs, newYs) {
        // new `StatSmooth` dataset here
        area {
            // use `Stat.*` columns for mappings
            x(Stat.x)
            y(Stat.y)
        }
    }
    points {
        x(newXs)
        y(newYs)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="fT5DvL"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2937047488152946,0.33650128451897965,0.3797455816136046,0.42354563543921,0.4678945636265345,0.5128073363728904,0.558297089068434,0.6043773005821542,0.6510578781227719,0.6983497864077173,0.746265085678759,0.7948163936812662,0.8440164285356331,0.893879840857105,0.9444272039590156,0.9956833388665468,1.0476732479777555,1.1004187202150673,1.1539383308678743,1.2082713816177024,1.2634398257267851,1.3186752026989308,1.3726488449352559,1.4291843891616267,1.4924165460508292,1.5657560769750394,1.6388718996770786,1.7289054341939132,1.8280471664533435,1.9162643338598284,2.0037618577709604,2.1079707722462375,2.240791344886155,2.3920780663331875,2.5461739829903074,2.6874221412604844,2.80016558754669,2.868747368251897,2.8775105297790744,2.8107981185311948,2.733200341857415,2.761857937365833,2.791033538863045,2.848660959057968,2.871477888279696,2.891525902732528,2.8989650945345944,2.897979234737794,2.9011917185024316,2.878775067783666,2.8009018045366556,2.933606636098272,2.9134761872657333,2.8765297520810993,2.8515550397148526,2.813303020111254,2.754719271087435,2.688248905670626,2.6271848427667583,2.6015030291520764,2.663003621581061,2.700538198666403,2.616312848517819,2.5125493986625083,2.4175964692968552,2.326768776034526,2.2351545226955154,2.1378237041498482,2.0438724973278126,1.94849678351387,1.8574105496726836,1.7672780983669603,1.678681067757626,1.5941387129370594,1.5160532248042644,1.444134317810646,1.3750923444947607,1.311458958074392,1.255341746858799,1.2012125013649293,1.1479883034004708,1.0952438832211202,1.04276635392464,0.9905336733937705,0.9385458299197853,0.8868105020604362,0.8353619343627221,0.7842267795446626,0.7334328587576175,0.6830087626605534,0.6329813280059671,0.5833705084120456,0.5341945367133992,0.48547164574463747,0.4372159367597067,0.3894249846898976,0.34209223288583757,0.2952178829185437,0.24885399541843073,0.20279375004651223]
},
"sampling":"none",
"position":"identity",
"geom":"area"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.92,-1.9,-

In [6]:
df.plot {
    statSmooth(speed, efficiency, method = SmoothMethod.Polynomial(2), smootherPointCount = 250) {
        ribbon {
            x(Stat.x)
            yMin(Stat.yMin)
            yMax(Stat.yMax)
        }
    }
    // Dataset is not changed here
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Q425pz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.3900003780509719,0.6920661389709915,0.5961064587171689,0.492460781141674,0.6302866130641619,0.6685197022731024,0.5354932283507859,0.6440931478309612,0.6830466536994991,0.6625888098665921,1.0150478691241638,0.8561740417434553,0.969773898738597,1.0483476345686966,1.106909081956581,0.9964850813189509,1.2737515571619076,1.2736462890264408,1.341092428244106,1.5442929750009913,1.7837618356501765,1.8386193445232522,2.861567785839137,3.025465637687313,3.36211893164043,3.194190215772683,3.4284703808330743,3.5867073521372466,3.6601647018299373,3.583521050175768,3.801088512218533,4.02371381878221,4.0708409689511305,4.021101553050831,4.0070676915215175,3.806149747093892,3.870145276270547,3.1954722760911394,3.0152624950122613,2.9052302055025256,2.8445245715518674,2.186435078291433,2.066155567291774,2.02684874559005,1.9515323987600302,1.8753443036792858,1.680734368571243,1.644722708663077,1.6204755676497942,1.228766136828285,1.248360538348174,1.125620582803132,1.1456840451268469,1.1280876708996,1.0317802013838202,0.9919891696401153,0.9641961128151121,0.8917430655934132,0.8832493928751232,0.9013180178539185,0.8448217251103447,0.6371847092189994,0.6677242759757861,0.6733889989612662,0.5963343356105675,0.40686546249591077,0.3420340342189992],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"yMin",
"ymax":"yMax"
},
"stat":"identity",
"data":{
"yMin":[-0.6129269452093076,-0.5591350715431291,-0.505775061016854,-0.45284698468216966,-0.40035091518068033,-0.3482869267834551,-0.29665509543155777,-0.24545549877755657,-0.19468821622804533,-0.1443533289871849,-0.09445092010127648,-0.04498107450439082,0.0040561209349362914,0.05266057736593682,0.10083220390674447,0.14857090759443697,0.19587659333395968,0.2427491638459372,0.28918851961335634,0.33519455882711946,0.38076717733047577,0.42590626856230046,0.47061172349926783,0.5148834305968819,0.5587212757293991,0.6021251421286422,0.645094910321722,0.6876304580676882,0.729731660293132,0.7713983890267606,0.812630513332987,0.8534278992445605,0.8937904096942956,0.9337179044459322,0.9732102400241986,1.0122672696441302,1.0508888431397174,1.0890748068919711,1.126825003756476,1.1641392729905502,1.2010174501801028,1.237459367166316,1.273464851972286,1.3090337287297493,1.3441658176060738,1.3788609347316623,1.4131188921279614,1.4469394976362826,1.4803225548476313,1.5132678630337995,1.545775217079952,1.577844407418988,1.6094752199679636,1.6406674360668763,1.671420832420154,1.7017351810411907,1.7316102492002998,1.7610457993765019,1.7900415892135366,1.8185973714805765,1.8467128940380886,1.874387899809351,1.901622126758145,1.9284153078731592,1.9547671711596861,1.9806774396392042,2.0061458313574603,2.031172059401698,2.0557558319276983,2.0798968521973213,2.103594818627241,2.1268494248496093,2.149660359785384,2.172027307731053,2.1939499484595357,2.2154279573360016,2.2364610054493785,2.257048759760302,2.2771908832662553,2.29688703518463,2.3161368711544115,2.334940043457178,2.353296201258045,2.371204990867162,2.388666056022301,2.4056790381930195,2.422243576906812,2.438359310097577,2.454025874476634,2.469242905926435,2.484010039916988,2.4

### `smoothLine` layer

`smoothLine` layer is a shortcut for fast plotting a smoothed line:

In [7]:
val smoothLineLayerPlot = plot {
    smoothLine(newXs, newYs)
    layout.title = "`smoothLine()` layer"
}
smoothLineLayerPlot

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7oJj6I"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2937047488152946,0.33650128451897965,0.3797455816136046,0.42354563543921,0.4678945636265345,0.5128073363728904,0.558297089068434,0.6043773005821542,0.6510578781227719,0.6983497864077173,0.746265085678759,0.7948163936812662,0.8440164285356331,0.893879840857105,0.9444272039590156,0.9956833388665468,1.0476732479777555,1.1004187202150673,1.1539383308678743,1.2082713816177024,1.2634398257267851,1.3186752026989308,1.3726488449352559,1.4291843891616267,1.4924165460508292,1.5657560769750394,1.6388718996770786,1.7289054341939132,1.8280471664533435,1.9162643338598284,2.0037618577709604,2.1079707722462375,2.240791344886155,2.3920780663331875,2.5461739829903074,2.6874221412604844,2.80016558754669,2.868747368251897,2.8775105297790744,2.8107981185311948,2.733200341857415,2.761857937365833,2.791033538863045,2.848660959057968,2.871477888279696,2.891525902732528,2.8989650945345944,2.897979234737794,2.9011917185024316,2.878775067783666,2.8009018045366556,2.933606636098272,2.9134761872657333,2.8765297520810993,2.8515550397148526,2.813303020111254,2.754719271087435,2.688248905670626,2.6271848427667583,2.6015030291520764,2.663003621581061,2.700538198666403,2.616312848517819,2.5125493986625083,2.4175964692968552,2.326768776034526,2.2351545226955154,2.1378237041498482,2.0438724973278126,1.94849678351387,1.8574105496726836,1.7672780983669603,1.678681067757626,1.5941387129370594,1.5160532248042644,1.444134317810646,1.3750923444947607,1.311458958074392,1.255341746858799,1.2012125013649293,1.1479883034004708,1.0952438832211202,1.04276635392464,0.9905336733937705,0.9385458299197853,0.8868105020604362,0.8353619343627221,0.7842267795446626,0.7334328587576175,0.6830087626605534,0.6329813280059671,0.5833705084120456,0.5341945367133992,0.48547164574463747,0.4372159367597067,0.3894249846898976,0.34209223288583757,0.2952178829185437,0.24885399541843073,0.20279375004651223]
},
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotCon

In [8]:
// Compare it with `statSmooth` + usual `line`
val statSmoothAndLinePlot = plot {
    statSmooth(newXs, newYs) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
    layout.title = "`statSmooth()` + non-statistical `line` layer"
}
plotGrid(listOf(smoothLineLayerPlot, statSmoothAndLinePlot))

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="QxrLjR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":2,
"nrow":1,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"`smoothLine()` layer"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-1.98,-1.94,-1.9,-1.8599999999999999,-1.82,-1.78,-1.74,-1.7,-1.66,-1.62,-1.58,-1.54,-1.5,-1.46,-1.42,-1.38,-1.3399999999999999,-1.2999999999999998,-1.26,-1.22,-1.18,-1.1400000000000001,-1.1,-1.06,-1.02,-0.98,-0.94,-0.8999999999999999,-0.8599999999999999,-0.8200000000000001,-0.78,-0.74,-0.7,-0.6599999999999999,-0.6199999999999999,-0.5799999999999998,-0.54,-0.5,-0.45999999999999996,-0.41999999999999993,-0.3799999999999999,-0.33999999999999986,-0.30000000000000004,-0.26,-0.21999999999999997,-0.17999999999999994,-0.1399999999999999,-0.09999999999999987,-0.06000000000000005,-0.020000000000000018,0.020000000000000018,0.06000000000000005,0.10000000000000009,0.14000000000000012,0.18000000000000016,0.2200000000000002,0.26000000000000023,0.30000000000000027,0.33999999999999986,0.3799999999999999,0.41999999999999993,0.45999999999999996,0.5,0.54,0.5800000000000001,0.6200000000000001,0.6600000000000001,0.7000000000000002,0.7400000000000002,0.7800000000000002,0.8200000000000003,0.8599999999999999,0.8999999999999999,0.94,0.98,1.02,1.06,1.1,1.1400000000000001,1.1800000000000002,1.2200000000000002,1.2600000000000002,1.3000000000000003,1.3400000000000003,1.38,1.42,1.46,1.5,1.54,1.58,1.62,1.6600000000000001,1.7000000000000002,1.7400000000000002,1.7800000000000002,1.8200000000000003,1.8599999999999999,1.9,1.94,1.98],
"y":[0.2937047488152946,0.33650128451897965,0.3797455816136046,0.42354563543921,0.4678945636265345,0.5128073363728904,0.558297089068434,0.6043773005821542,0.6510578781227719,0.6983497864077173,0.746265085678759,0.7948163936812662,0.8440164285356331,0.893879840857105,0.9444272039590156,0.9956833388665468,1.0476732479777555,1.1004187202150673,1.1539383308678743,1.2082713816177024,1.2634398257267851,1.3186752026989308,1.3726488449352559,1.4291843891616267,1.4924165460508292,1.5657560769750394,1.6388718996770786,1.7289054341939132,1.8280471664533435,1.9162643338598284,2.0037618577709604,2.1079707722462375,2.240791344886155,2.3920780663331875,2.5461739829903074,2.6874221412604844,2.80016558754669,2.868747368251897,2.8775105297790744,2.8107981185311948,2.733200341857415,2.761857937365833,2.791033538863045,2.848660959057968,2.871477888279696,2.891525902732528,2.8989650945345944,2.897979234737794,2.9011917185024316,2.878775067783666,2.8009018045366556,2.933606636098272,2.9134761872657333,2.8765297520810993,2.8515550397148526,2.813303020111254,2.754719271087435,2.688248905670626,2.6271848427667583,2.6015030291520764,2.663003621581061,2.700538198666403,2.616312848517819,2.5125493986625083,2.4175964692968552,2.326768776034526,2.2351545226955154,2.1378237041498482,2.0438724973278126,1.94849678351387,1.8574105496726836,1.7672780983669603,1.678681067757626,1.5941387129370594,1.5160532248042644,1.444134317810646,1.3750923444947607,1.311458958074392,1.255341746858799,1.2012125013649293,1.1479883034004708,1.0952438832211202,1.04276635392464,0.9905336733937705,0.9385458299197853,0.8868105020604362,0.8353619343627221,0.7842267795446626,0.7334328587576175,0.6830087626605534,0.6329813280059671,0.5833705084120456,0.5341945367133992,0.48547164574463747,0.4372159367597067,0.3894249846898976,0.34209223288583757,0.2952178829185437,0.24885399541843073,0.20279375

`smoothLine` uses `statSmooth` and `line` and performs coordinate mappings under the hood.
And we can customize `smoothLine` layer: `smoothLine()` optionally opens a new context,
where we can configure the line (as in the usual context opened by `line { ... }`) —
even change coordinate mappings from default ones.
`StatSmooth` dataset of `smoothLine` is also can be accessed here.

In [9]:
df.plot {
    smoothLine(speed, efficiency, SmoothMethod.LOESS(span = 0.1), 120) {
        // change a column mapped on `y` to `Stat.scaled`
        y(Stat.yMax)
        color = Color.RED
        width = 4.0
    }
    points {
        x(speed)
        y(efficiency)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="N9ueg5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"efficiency":[0.3900003780509719,0.6920661389709915,0.5961064587171689,0.492460781141674,0.6302866130641619,0.6685197022731024,0.5354932283507859,0.6440931478309612,0.6830466536994991,0.6625888098665921,1.0150478691241638,0.8561740417434553,0.969773898738597,1.0483476345686966,1.106909081956581,0.9964850813189509,1.2737515571619076,1.2736462890264408,1.341092428244106,1.5442929750009913,1.7837618356501765,1.8386193445232522,2.861567785839137,3.025465637687313,3.36211893164043,3.194190215772683,3.4284703808330743,3.5867073521372466,3.6601647018299373,3.583521050175768,3.801088512218533,4.02371381878221,4.0708409689511305,4.021101553050831,4.0070676915215175,3.806149747093892,3.870145276270547,3.1954722760911394,3.0152624950122613,2.9052302055025256,2.8445245715518674,2.186435078291433,2.066155567291774,2.02684874559005,1.9515323987600302,1.8753443036792858,1.680734368571243,1.644722708663077,1.6204755676497942,1.228766136828285,1.248360538348174,1.125620582803132,1.1456840451268469,1.1280876708996,1.0317802013838202,0.9919891696401153,0.9641961128151121,0.8917430655934132,0.8832493928751232,0.9013180178539185,0.8448217251103447,0.6371847092189994,0.6677242759757861,0.6733889989612662,0.5963343356105675,0.40686546249591077,0.3420340342189992],
"speed":[-1.98,-1.92,-1.9,-1.88,-1.78,-1.74,-1.7,-1.64,-1.52,-1.48,-1.34,-1.3,-1.24,-1.22,-1.18,-1.16,-1.0,-0.96,-0.94,-0.86,-0.84,-0.74,-0.42,-0.4,-0.36,-0.34,-0.32,-0.24,-0.2,-0.16,-0.14,0.02,0.04,0.06,0.08,0.14,0.2,0.34,0.38,0.46,0.5,0.64,0.66,0.72,0.74,0.76,0.8,0.82,0.96,1.04,1.06,1.08,1.1,1.14,1.16,1.2,1.24,1.26,1.38,1.42,1.46,1.48,1.58,1.74,1.88,1.9,1.98]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"speed",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"efficiency",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"yMax"
},
"stat":"identity",
"data":{
"yMax":[0.4469471593933701,0.49012356905092386,0.5318973859128949,0.5742964221207182,0.6104846067231845,0.6374482500508374,0.6573360963828749,0.670699558980538,0.6734329222974571,0.6739375883483714,0.6776863401991273,0.6826450675212489,0.688798111112376,0.697181756268267,0.708844358857645,0.7265655473842484,0.7552476929309075,0.7917588837082488,0.830638654882493,0.8664265166345708,0.8971440626698596,0.9467480291800194,1.0076455453654947,1.045209146302783,1.0781016757784636,1.0938881870099468,1.1247433378285134,1.1680091135224757,1.2160021046584135,1.2610389154268817,1.2979116749097643,1.364694171814443,1.4428799329374116,1.5051841587199333,1.5925544590404317,1.692259340957936,1.775900952858679,1.8491581854515005,1.9178949171784179,1.9862612276960385,2.0574937151416464,2.134820457791876,2.2214695396242967,2.320669048237242,2.4356470724921104,2.569631699878812,2.725851013612067,2.907458315343221,3.0667564262575477,3.1635721318449255,3.4140650512286634,3.5597420509436684,3.604928116438165,3.6801163047810292,3.682437309011651,3.6730816796164496,3.7517993708030275,3.818419843205029,3.8807305805571337,3.9483829169661386,4.031028137950632,4.100446896177263,4.0309814929079675,3.915487998034942,3.853555547873371,3.8585580609435244,3.827969321842662,3.7177849206695956,3.5561959674035104,3.3746834158212744,3.2043206195079197,3.0397971090543328,2.851663140222457,2.731133415677657,2.774402114862069,2.858652629420981,2.846993424206966,2.7354878682455257,2.5267955350692897,2.2241412804769243,1.8951619612186246,1.7914947391992906,1.9540162951301219,1.807905957953859,1.6694559515132115,1.

## Grouped `statSmoth`

`statSmooth` can be applied for grouped data —
statistics will be calculated on each group independently but with equal categories.
This application returns a new `GroupBy`
dataset with the same keys as the old one but with `StatSmooth` groups instead of old ones.

In [10]:
// Generate two lines
val fA = { x: Double -> 0.02 * x * x * x - 0.2 * x * x + 0.1 * x + 2.1 }
val fB = { x: Double -> -0.1 * x * x * x + 0.5 * x * x - 0.8 }
val xRange = (-500..500).map { it / 100.0 }
val noisesA = NormalDistribution(0.0, 0.05).sample(xRange.size).toList()
val noisesB = NormalDistribution(0.0, 0.2).sample(xRange.size).toList()
val valuesA = xRange.zip(noisesA).map { fA(it.first) + it.second }
val valuesB = xRange.zip(noisesB).map { fB(it.first) + it.second }

val (xsA, ysA) = xRange.zip(valuesA).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()
val (xsB, ysB) = xRange.zip(valuesB).shuffled(Random(17)).take(xRange.size * 1 / 3).sortedBy { it.first }
    .unzip()

In [11]:
// Gather them into `DataFrame` with "A" and "B" keys in "category" column
val valuesDF = dataFrameOf(
    "time" to xsA + xsB,
    "value" to ysA + ysB,
    "category" to List(xsA.size) { "A" } + List(xsB.size) { "B" }
)
valuesDF.head(5)

<iframe onload="o_resize_iframe_out_3()" style="width:100%;" class="result_container" id="iframe_out_3" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1073741826"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for 

It has the following signature:

| time | value | category |
|------|-------|----------|

In [12]:
// Group it by "category"
val groupedDF = valuesDF.groupBy { category }
groupedDF

<iframe onload="o_resize_iframe_out_4()" style="width:100%;" class="result_container" id="iframe_out_4" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1073741827"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++

Now we have a `GroupBy` with a signature

<table>
  <thead>
    <tr>
      <th>key: [category]</th>
      <th>group: DataFrame[time|value|category]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>A</td>
      <td>A-Group</td>
    </tr>
    <tr>
      <td>B</td>
      <td>B-Group</td>
    </tr>
  </tbody>
</table>

In [13]:
groupedDF.statSmooth { x(time); y(value) }

<iframe onload="o_resize_iframe_out_5()" style="width:100%;" class="result_container" id="iframe_out_5" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_1073741830"></table>

<p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0; i < df.cols.length; i++

After `statSmooth` applying it's still a `GroupBy` but with different signature of `group` —
all groups have the same signature as usual `DataFrame` after `statSmooth` applying (i.e. `StatSmoothFrame`):

<table>
  <thead>
    <tr>
      <th>key: [drv]</th>
      <th>group: StaSmoothFrame</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>"A"</td>
      <td>"A"-Group</td>
    </tr>
    <tr>
      <td>"B"</td>
      <td>"B"-Group</td>
    </tr>
  </tbody>
</table>

As you can see, we did indeed do a `statSmooth` transformation within groups, the grouping keys did not change.

The plotting process doesn't change much — we do everything the same.

In [14]:
groupedDF.plot {
    statSmooth(time, value) {
        line {
            x(Stat.x)
            y(Stat.y)
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="aFLsPd"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,

As you can see, there are two lines because we have two groups of data.
To distinguish them, we need to add mapping to the color from the key.
This is convenient — the key is available in the context

In [15]:
groupedDF.plot {
    statSmooth(time, value, method = SmoothMethod.Polynomial(3)) {
        line {
            x(Stat.x)
            y(Stat.y)
            color(key.category)
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Z47U0O"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.740606060606060

The `smoothLine()` layer also works.
Moreover, if we have exactly one grouping key, a mapping from it to `color` will be created by default.

In [16]:
groupedDF.plot {
    smoothLine(time, value)
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="yJ27Rh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-

We can customize it like we used to. From the differences — access to `key` columns:

In [17]:
groupedDF.plot {
    smoothLine(time, value) {
        color = Color.GREEN
        type(key.category)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="52IWDH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"linetype"
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"linetype":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454

### Inside `groupBy{}` plot context

We can apply `groupBy` modification to the initial dataset and build a histogram with grouped data the same way:

In [18]:
valuesDF.plot {
    groupBy(category) {
        smoothLine(time, value)
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GC8Myv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"time",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"value",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y",
"color":"category",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"&merged_groups":["A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","A","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B","B"],
"x":[-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-2.243636363636363,-2.1430303030303026,-2.042424242424242,-1.9418181818181814,-1.8412121212121209,-1.7406060606060603,-1.6399999999999997,-1.539393939393939,-1.4387878787878785,-1.338181818181818,-1.2375757575757573,-1.1369696969696967,-1.0363636363636362,-0.9357575757575756,-0.835151515151515,-0.7345454545454544,-0.6339393939393938,-0.5333333333333332,-0.4327272727272726,-0.33212121212121204,-0.23151515151515145,-0.13090909090908998,-0.030303030303029388,0.0703030303030312,0.1709090909090918,0.2715151515151524,0.37212121212121296,0.47272727272727355,0.5733333333333341,0.6739393939393947,0.7745454545454553,0.8751515151515159,0.9757575757575765,1.076363636363637,1.1769696969696977,1.2775757575757583,1.3781818181818188,1.4787878787878794,1.57939393939394,1.6800000000000006,1.7806060606060612,1.8812121212121218,1.9818181818181824,2.082424242424243,2.1830303030303035,2.283636363636364,2.3842424242424247,2.4848484848484853,2.585454545454546,2.6860606060606065,2.786666666666667,2.8872727272727277,2.9878787878787882,3.088484848484849,3.1890909090909103,3.28969696969697,3.3903030303030315,3.490909090909091,3.5915151515151527,3.6921212121212124,3.792727272727274,3.8933333333333335,3.993939393939395,4.094545454545455,4.195151515151516,4.295757575757576,4.396363636363637,4.496969696969697,4.5975757575757585,4.69818181818182,4.79878787878788,4.899393939393941,5.000000000000001,-4.96,-4.859393939393939,-4.758787878787879,-4.658181818181818,-4.557575757575758,-4.456969696969697,-4.356363636363636,-4.255757575757576,-4.155151515151515,-4.054545454545455,-3.953939393939394,-3.8533333333333335,-3.7527272727272725,-3.652121212121212,-3.5515151515151513,-3.4509090909090907,-3.35030303030303,-3.2496969696969695,-3.149090909090909,-3.0484848484848484,-2.9478787878787878,-2.847272727272727,-2.7466666666666666,-2.646060606060606,-2.545454545454545,-2.4448484848484844,-2.344242424242424,-